## Table detection and Table data extraction from Invoices

* Extracting table information from invoices is a necessary task for many companies. Lot of human effort is need to achieve it. The automation of this task will save human effort, time and money for companies. 
* To automate this process, the first challenge is to detect the table in an invoice image, which is not easy as the invoices differ based on the company and its purpose. And few invoices have structured and borderd tables and the other invoices may contain line by line data which looks structured like table. So, the program should first try to identify the table, and then should extract rows data from it. Borders tables might be identified with the line pattens, but to detect the unstructured and non-bordered tables we should write a program that understands the structure of the data, like we should train the program with more invoices that contains the labelled tables.
* As the power Artificial intelligence is growing time to time, there are good techniques that will learn from given data and can identify similar objects from the unseen images. These object detection algorithms have the Convolution Nueral networks as its base form, will be trained with the table like structure, as the data is more and diverse the learning capability increases. There are lot of variants of CNN to understand and detect objects in an image, we will analyse those soon.
* Once the table region is detected, that table image will be fed to table data extraction function, which is our custom function to detect columns and rows of table, and we also identify the text inside each table cell using tessaract-ocr which is a popular tool or library from google.